<a href="https://colab.research.google.com/github/sunil-karki/NLP-Question_Classification/blob/master/NLP-DisasterTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sunilkarki520","key":"8368d7fdc07a580b54aeb9a6df65cda4"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 63.3MB/s]
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 137MB/s]
  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 23.3MB/s]


In [5]:
!ls

kaggle.json  sample_data  sample_submission.csv  test.csv  train.csv


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate

from keras.optimizers import Adam
from keras import backend as K      ######
from keras.models import Model
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

In [37]:

train_df = pd.read_csv('train.csv') 
test_df = pd.read_csv('test.csv') 
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

Train shape :  (7613, 5)
Test shape :  (3263, 4)


In [38]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [9]:
test_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [0]:

train_data = train_df 
train_data = train_data.replace(np.nan, '', regex=True)

train_data['text'] = train_data['text'] + ' ' + train_data['location'] + ' ' + train_data['keyword']
del train_data['keyword']
del train_data['location']
#train_data.iloc[0]
a = train_data.iloc[0][['text']].values
#print(a)

test_data = test_df 
test_data = test_data.replace(np.nan, '', regex=True)

test_data['text'] = test_data['text'] + ' ' + test_data['location'] + ' ' + test_data['keyword']

In [55]:
train_data

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...
7608,10869,Two giant cranes holding a bridge collapse int...,1
7609,10870,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,Police investigating after an e-bike collided ...,1


In [0]:
## some config values 
embed_size = 300 # how big is each word vector
max_features = 95000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 70 # max number of words in a question to use

## ValueError: Layer weight shape (95000, 300) not compatible with provided weight shape (24568, 300)
max_features = 24568

In [0]:
maxlen = max([len(data) for data in train_data['text'].values])   ##maximum number of words in column text
print(maxlen)    ##195

In [0]:
## split to train and val
train_data, val_data = train_test_split(train_data, test_size=0.001, random_state=2020) 
 
## fill up the missing values
train_X = train_data['text'].values
val_X = val_data['text'].values
test_X = test_data['text'].values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

## Get the target values
train_y = train_data['target'].values   #train_y is an array i.e. array([1, 0, 1, ..., 1, 1, 1])
val_y = val_data['target'].values  
    
#shuffling the data
np.random.seed(2018)
trn_idx = np.random.permutation(len(train_X))
val_idx = np.random.permutation(len(val_X))

train_X = train_X[trn_idx]  #train_X is an array i,e. array([1653, 2280, 1170, ..., 6370, 2662, 1274])
val_X = val_X[val_idx]
train_y = train_y[trn_idx]
val_y = val_y[val_idx]    

In [124]:
word_index = tokenizer.word_index
print(word_index)

{'co': 1, 't': 2, 'http': 3, 'the': 4, 'a': 5, 'in': 6, 'to': 7, 'of': 8, 'and': 9, 'i': 10, 'is': 11, 'for': 12, 'on': 13, 'you': 14, 'my': 15, 'with': 16, 'it': 17, 'that': 18, 'at': 19, 'by': 20, 'this': 21, 'new': 22, 'from': 23, 'https': 24, 'be': 25, 'are': 26, 'was': 27, 'have': 28, 'like': 29, 'amp': 30, 'me': 31, 'as': 32, 'up': 33, 'but': 34, 'just': 35, 'so': 36, 'fire': 37, 'not': 38, 'your': 39, 'usa': 40, 'out': 41, 'no': 42, 'all': 43, 'emergency': 44, 'will': 45, 'after': 46, 'an': 47, 'has': 48, "i'm": 49, 'when': 50, 'body': 51, 'if': 52, 'we': 53, '2': 54, 'get': 55, 'via': 56, 'now': 57, 'or': 58, 'more': 59, 'about': 60, 'suicide': 61, 'news': 62, 'they': 63, 'california': 64, 'he': 65, 'people': 66, 'what': 67, 'one': 68, 'burning': 69, 'how': 70, 'been': 71, 'disaster': 72, 'over': 73, 'who': 74, "it's": 75, 'police': 76, 'buildings': 77, 'into': 78, "don't": 79, 'nuclear': 80, 'do': 81, 'ca': 82, "'": 83, 'video': 84, 'can': 85, 'york': 86, 'there': 87, 'u': 88,

In [125]:
train_X

array([[    0,     0,     0, ...,  1862,   471,   392],
       [    0,     0,     0, ...,   859,   304,   620],
       [    0,     0,     0, ..., 10261,   456,   170],
       ...,
       [    0,     0,     0, ..., 21834,  7206,   143],
       [    0,     0,     0, ...,   353, 13500,   173],
       [    0,     0,     0, ...,  3384,  3385,   174]], dtype=int32)

In [72]:
!kaggle competitions download -c quora-insincere-questions-classification

 89% 49.0M/54.9M [00:01<00:00, 30.2MB/s]
100% 54.9M/54.9M [00:01<00:00, 49.4MB/s]
  0% 0.00/4.09M [00:00<?, ?B/s]
100% 4.09M/4.09M [00:00<00:00, 67.2MB/s]
100% 5.96G/5.96G [01:29<00:00, 47.3MB/s]
100% 5.96G/5.96G [01:29<00:00, 71.7MB/s]
 57% 9.00M/15.8M [00:00<00:00, 25.8MB/s]
100% 15.8M/15.8M [00:00<00:00, 40.0MB/s]


In [73]:
!ls

embeddings.zip	sample_submission.csv	   test.csv.zip
kaggle.json	sample_submission.csv.zip  train.csv
sample_data	test.csv		   train.csv.zip


In [74]:
!unzip embeddings.zip

Archive:  embeddings.zip
   creating: GoogleNews-vectors-negative300/
   creating: glove.840B.300d/
   creating: paragram_300_sl999/
   creating: wiki-news-300d-1M/
  inflating: glove.840B.300d/glove.840B.300d.txt  
  inflating: GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin  
  inflating: wiki-news-300d-1M/wiki-news-300d-1M.vec  
  inflating: paragram_300_sl999/README.txt  
  inflating: paragram_300_sl999/paragram_300_sl999.txt  


In [75]:
!ls

embeddings.zip			sample_data		   train.csv
glove.840B.300d			sample_submission.csv	   train.csv.zip
GoogleNews-vectors-negative300	sample_submission.csv.zip  wiki-news-300d-1M
kaggle.json			test.csv
paragram_300_sl999		test.csv.zip


In [0]:
def load_glove(word_index):
    # EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    EMBEDDING_FILE = 'glove.840B.300d/glove.840B.300d.txt'
    ## This file contains content as following
    #Glass -0.41236 0.35532 -0.076236 0.43268 0.14769 -1.1381 -0.30427 0.37039 -0.3804 0.029443 -0.081454 0.15791 -0.32558 -0.35445 0.11918 -0.13342 -0.087387 1.2125 -0.048223  .....
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 

In [127]:
embedding_matrix_1 = load_glove(word_index)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [0]:
# https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [0]:
def model_gru_atten_3(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    #x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    #x = Bidirectional(CuDNNGRU(100, return_sequences=True))(x)
    #x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    x = Bidirectional(GRU(100, return_sequences=True))(x)
    x = Bidirectional(GRU(64, return_sequences=True))(x)
    
    x = Attention(maxlen)(x)        ## Finding on this
    
    x = Dense(1, activation="sigmoid")(x)
    #x = Dense(1, activation="softmax")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [0]:
def train_pred(model, epochs_param):
    model.fit(train_X, train_y, batch_size=512, epochs = epochs_param, validation_data=(val_X, val_y))
    pred_val_y = model.predict([val_X], batch_size=1024, verbose=1)
    pred_test_y = model.predict([test_X], batch_size=1024, verbose=1)
    return pred_val_y, pred_test_y, model

In [0]:
##  ValueError: Layer weight shape (95000, 300) not compatible with provided weight shape (24568, 300)

In [149]:
pred_val_y, pred_test_y, model = train_pred(model_gru_atten_3(embedding_matrix_1), epochs_param = 10)

Train on 7605 samples, validate on 8 samples
Epoch 1/10
7605/7605 [==============================] - 113s 15ms/step - loss: 0.7379 - acc: 0.5487 - val_loss: 0.6875 - val_acc: 0.3750
Epoch 2/10
7605/7605 [==============================] - 100s 13ms/step - loss: 0.6287 - acc: 0.6651 - val_loss: 0.6004 - val_acc: 0.7500
Epoch 3/10
7605/7605 [==============================] - 98s 13ms/step - loss: 0.5193 - acc: 0.7582 - val_loss: 0.3866 - val_acc: 0.8750
Epoch 4/10
7605/7605 [==============================] - 98s 13ms/step - loss: 0.4511 - acc: 0.7971 - val_loss: 0.2928 - val_acc: 0.8750
Epoch 5/10
7605/7605 [==============================] - 99s 13ms/step - loss: 0.4211 - acc: 0.8145 - val_loss: 0.2966 - val_acc: 0.8750
Epoch 6/10
7605/7605 [==============================] - 100s 13ms/step - loss: 0.4052 - acc: 0.8237 - val_loss: 0.2908 - val_acc: 0.8750
Epoch 7/10
7605/7605 [==============================] - 100s 13ms/step - loss: 0.3909 - acc: 0.8310 - val_loss: 0.3369 - val_acc: 0.8750

In [151]:
pred_test_y

array([[0.45894313],
       [0.8879403 ],
       [0.61471725],
       ...,
       [0.9388808 ],
       [0.83187866],
       [0.10463531]], dtype=float32)

In [150]:
score = model.evaluate(val_X, val_y, verbose=0)
print('test loss, test acc:', results)

NameError: ignored

In [163]:
pred_val_y_a = (pred_val_y > 0.33).astype(int)
print(pred_val_y_a.shape)

(8, 1)


In [162]:
val_y

array([0, 0, 1, 1, 1, 0, 0, 0])

In [161]:
check = np.equal(pred_val_y_a, val_y)
#check_1 = np.mean(check, axis=0)
print(check)

[[ True  True False False False  True  True  True]
 [ True  True False False False  True  True  True]
 [False False  True  True  True False False False]
 [False False  True  True  True False False False]
 [False False  True  True  True False False False]
 [False False  True  True  True False False False]
 [ True  True False False False  True  True  True]
 [ True  True False False False  True  True  True]]


In [158]:
check = np.where( np.equal(pred_val_y_a, val_y) == True, 1, 0) 
#check_1 = np.mean(check, axis=0)
print(check)

[[1 1 0 0 0 1 1 1]
 [1 1 0 0 0 1 1 1]
 [0 0 1 1 1 0 0 0]
 [0 0 1 1 1 0 0 0]
 [0 0 1 1 1 0 0 0]
 [0 0 1 1 1 0 0 0]
 [1 1 0 0 0 1 1 1]
 [1 1 0 0 0 1 1 1]]


In [0]:
for i, predicted in enumerate(predictions):
    if predicted[0] > 0.25:
        print "bigger than 0.25"
        #assign i to class 1
    else:
        print "smaller than 0.25"
        #assign i to class 0

In [0]:
a = pred_test_y

In [144]:
b = a.astype(int)  #numpy
print(b)

<class 'numpy.ndarray'>


In [145]:
unique_elements, counts_elements = np.unique(b, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

[[   0]
 [3263]]
